In [1]:
import pandas as pd
challenger_20=pd.read_csv('D:/Final_Project/data/lol_elo(2020)/Challenger_Ranked_Games.csv')
challenger_21=pd.read_csv('D:/Final_Project/data/match_id.csv',index_col=0)

In [2]:
challenger_21

,platformId,gameId,champion,queue,season,timestamp,role,lane
0,KR,5353779756,142,420,13,1627408794568,SOLO,MID
1,KR,5351903351,142,420,13,1627326705828,DUO_SUPPORT,NONE
2,KR,5351960180,7,420,13,1627323820506,DUO,MID
3,KR,5351809074,142,420,13,1627321901323,DUO,TOP
4,KR,5351805859,142,420,13,1627319548112,DUO,MID
...,...,...,...,...,...,...,...,...
29698,KR,5341946033,58,420,13,1626969312953,DUO,TOP
29699,KR,5341857786,58,420,13,1626966292011,SOLO,TOP
29700,KR,5341779870,875,420,13,1626963781917,SOLO,TOP
29701,KR,5341668681,126,420,13,1626960601379,SOLO,TOP


In [3]:
challenger_20

,gameId,gameDuraton,blueWins,blueFirstBlood,blueFirstTower,blueFirstBaron,blueFirstDragon,blueFirstInhibitor,blueDragonKills,blueBaronKills,...,redAssist,redChampionDamageDealt,redTotalGold,redTotalMinionKills,redTotalLevel,redAvgLevel,redJungleMinionKills,redKillingSpree,redTotalHeal,redObjectDamageDealt
0,4247263043,1323,0,1,0,0,0,0,0,0,...,68,73016,49832,533,64,12.8,138,8,38711,60463
1,4247155821,1317,1,0,0,0,1,0,2,0,...,8,44218,37136,571,59,11.8,134,3,24829,14105
2,4243963257,932,0,0,0,0,0,0,0,0,...,20,30797,35364,400,50,10.0,89,5,12589,45201
3,4241678498,2098,0,0,0,1,1,0,1,1,...,72,125866,70528,832,84,16.8,205,9,65312,88202
4,4241538868,2344,1,1,1,1,1,1,2,2,...,83,131536,74596,794,85,17.0,227,7,86431,64141
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26899,4143231833,1468,0,1,0,0,0,0,1,0,...,40,75531,46558,478,64,12.8,141,6,48596,42733
26900,4157911901,970,0,1,1,0,1,0,3,0,...,31,99837,70177,455,72,14.4,136,11,15817,50217
26901,3764171638,1480,0,1,1,0,1,0,1,0,...,34,55214,48849,636,66,13.2,175,6,25448,27675
26902,4110201724,1837,1,1,0,0,1,0,3,0,...,29,88366,52625,660,70,14.0,168,5,25003,45455


In [4]:
challenger_20.duplicated(['gameId']).sum()

56

In [5]:
challenger_21.duplicated(['gameId']).sum()

16329

In [6]:
chall_20_gameId=pd.DataFrame(challenger_20['gameId'])
chall_21_gameId=pd.DataFrame(challenger_21['gameId'])

In [7]:
chall_20_gameId.drop_duplicates(['gameId'],inplace=True)
chall_20_gameId.reset_index(inplace=True)
chall_20_gameId.drop('index',1,inplace=True)
chall_20_gameId

,gameId
0,4247263043
1,4247155821
2,4243963257
3,4241678498
4,4241538868
...,...
26843,4143231833
26844,4157911901
26845,3764171638
26846,4110201724


In [8]:
chall_21_gameId.drop_duplicates(['gameId'],inplace=True)
chall_21_gameId.reset_index(inplace=True)
chall_21_gameId.drop('index',1,inplace=True)
chall_21_gameId

,gameId
0,5353779756
1,5351903351
2,5351960180
3,5351809074
4,5351805859
...,...
13369,5347379013
13370,5347476046
13371,5347434370
13372,5345526209


In [9]:
import time
import json
import requests
from tqdm import tqdm_notebook

api_key='변경함'

In [11]:
match_20_data=pd.DataFrame([])
for i in tqdm_notebook(range(len(chall_20_gameId))):
    api_url='https://kr.api.riotgames.com/lol/match/v4/matches/'+str(chall_20_gameId['gameId'][i]) + '?api_key=' + api_key
    r=requests.get(api_url)
    
    while r.status_code!=200: # 요청 제한 또는 오류로 인해 정상적으로 받아오지 않는 상태라면, 5초 간 시간을 지연
        time.sleep(10)
        r = requests.get(api_url)
        
    test=pd.DataFrame(r.json()['participants'])
    test_pre=pd.DataFrame({
    'play_time':r.json()['gameDuration'],
    'result':int(test['stats'][0]['win']),
    'blue_1_champ':test['championId'][0],
    'blue_1_spell1':test['spell1Id'][0],
    'blue_1_spell2':test['spell2Id'][0],
    'blue_1_kill':test['stats'][0]['kills'],
    'blue_1_death':test['stats'][0]['deaths'],
    'blue_1_assist':test['stats'][0]['assists'],
    'blue_1_totaldamage_dealt':test['stats'][0]['totalDamageDealt'],
    'blue_1_totaldamage_taken':test['stats'][0]['totalDamageTaken'],
    'blue_1_visionscore':test['stats'][0]['visionScore'],
    'blue_1_cs':test['stats'][0]['totalMinionsKilled'],   
    'blue_2_champ':test['championId'][1],
    'blue_2_spell1':test['spell1Id'][1],
    'blue_2_spell2':test['spell2Id'][1],
    'blue_2_kill':test['stats'][1]['kills'],
    'blue_2_death':test['stats'][1]['deaths'],
    'blue_2_assist':test['stats'][1]['assists'],
    'blue_2_totaldamage_dealt':test['stats'][1]['totalDamageDealt'],
    'blue_2_totaldamage_taken':test['stats'][1]['totalDamageTaken'],
    'blue_2_visionscore':test['stats'][1]['visionScore'],
    'blue_2_cs':test['stats'][1]['totalMinionsKilled'],    
    'blue_3_champ':test['championId'][2],
    'blue_3_spell1':test['spell1Id'][2],
    'blue_3_spell2':test['spell2Id'][2],
    'blue_3_kill':test['stats'][2]['kills'],
    'blue_3_death':test['stats'][2]['deaths'],
    'blue_3_assist':test['stats'][2]['assists'],
    'blue_3_totaldamage_dealt':test['stats'][2]['totalDamageDealt'],
    'blue_3_totaldamage_taken':test['stats'][2]['totalDamageTaken'],
    'blue_3_visionscore':test['stats'][2]['visionScore'],
    'blue_3_cs':test['stats'][2]['totalMinionsKilled'],
    'blue_4_champ':test['championId'][3],
    'blue_4_spell1':test['spell1Id'][3],
    'blue_4_spell2':test['spell2Id'][3],
    'blue_4_kill':test['stats'][3]['kills'],
    'blue_4_death':test['stats'][3]['deaths'],
    'blue_4_assist':test['stats'][3]['assists'],
    'blue_4_totaldamage_dealt':test['stats'][3]['totalDamageDealt'],
    'blue_4_totaldamage_taken':test['stats'][3]['totalDamageTaken'],
    'blue_4_visionscore':test['stats'][3]['visionScore'],
    'blue_4_cs':test['stats'][3]['totalMinionsKilled'],    
    'blue_5_champ':test['championId'][4],
    'blue_5_spell1':test['spell1Id'][4],
    'blue_5_spell2':test['spell2Id'][4],
    'blue_5_kill':test['stats'][4]['kills'],
    'blue_5_death':test['stats'][4]['deaths'],
    'blue_5_assist':test['stats'][4]['assists'],
    'blue_5_totaldamage_dealt':test['stats'][4]['totalDamageDealt'],
    'blue_5_totaldamage_taken':test['stats'][4]['totalDamageTaken'],
    'blue_5_visionscore':test['stats'][4]['visionScore'],
    'blue_5_cs':test['stats'][4]['totalMinionsKilled'],
    'red_1_champ':test['championId'][5],
    'red_1_spell1':test['spell1Id'][5],
    'red_1_spell2':test['spell2Id'][5],
    'red_1_kill':test['stats'][5]['kills'],
    'red_1_death':test['stats'][5]['deaths'],
    'red_1_assist':test['stats'][5]['assists'],
    'red_1_totaldamage_dealt':test['stats'][5]['totalDamageDealt'],
    'red_1_totaldamage_taken':test['stats'][5]['totalDamageTaken'],
    'red_1_visionscore':test['stats'][5]['visionScore'],
    'red_1_cs':test['stats'][5]['totalMinionsKilled'],
    'red_2_champ':test['championId'][6],
    'red_2_spell1':test['spell1Id'][6],
    'red_2_spell2':test['spell2Id'][6],
    'red_2_kill':test['stats'][6]['kills'],
    'red_2_death':test['stats'][6]['deaths'],
    'red_2_assist':test['stats'][6]['assists'],
    'red_2_totaldamage_dealt':test['stats'][6]['totalDamageDealt'],
    'red_2_totaldamage_taken':test['stats'][6]['totalDamageTaken'],
    'red_2_visionscore':test['stats'][6]['visionScore'],
    'red_2_cs':test['stats'][6]['totalMinionsKilled'],
    'red_3_champ':test['championId'][7],
    'red_3_spell1':test['spell1Id'][7],
    'red_3_spell2':test['spell2Id'][7],
    'red_3_kill':test['stats'][7]['kills'],
    'red_3_death':test['stats'][7]['deaths'],
    'red_3_assist':test['stats'][7]['assists'],
    'red_3_totaldamage_dealt':test['stats'][7]['totalDamageDealt'],
    'red_3_totaldamage_taken':test['stats'][7]['totalDamageTaken'],
    'red_3_visionscore':test['stats'][7]['visionScore'],
    'red_3_cs':test['stats'][7]['totalMinionsKilled'],
    'red_4_champ':test['championId'][8],
    'red_4_spell1':test['spell1Id'][8],
    'red_4_spell2':test['spell2Id'][8],
    'red_4_kill':test['stats'][8]['kills'],
    'red_4_death':test['stats'][8]['deaths'],
    'red_4_assist':test['stats'][8]['assists'],
    'red_4_totaldamage_dealt':test['stats'][8]['totalDamageDealt'],
    'red_4_totaldamage_taken':test['stats'][8]['totalDamageTaken'],
    'red_4_visionscore':test['stats'][8]['visionScore'],
    'red_4_cs':test['stats'][8]['totalMinionsKilled'],
    'red_5_champ':test['championId'][9],
    'red_5_spell1':test['spell1Id'][9],
    'red_5_spell2':test['spell2Id'][9],
    'red_5_kill':test['stats'][9]['kills'],
    'red_5_death':test['stats'][9]['deaths'],
    'red_5_assist':test['stats'][9]['assists'],
    'red_5_totaldamage_dealt':test['stats'][9]['totalDamageDealt'],
    'red_5_totaldamage_taken':test['stats'][9]['totalDamageTaken'],
    'red_5_visionscore':test['stats'][9]['visionScore'],
    'red_5_cs':test['stats'][9]['totalMinionsKilled']
    },index=[i])
    match_20_data=pd.concat([match_20_data,test_pre])
match_20_data

<ipython-input-11-594e26993347>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(chall_20_gameId))):


,play_time,result,blue_1_champ,blue_1_spell1,blue_1_spell2,blue_1_kill,blue_1_death,blue_1_assist,blue_1_totaldamage_dealt,blue_1_totaldamage_taken,...,red_5_champ,red_5_spell1,red_5_spell2,red_5_kill,red_5_death,red_5_assist,red_5_totaldamage_dealt,red_5_totaldamage_taken,red_5_visionscore,red_5_cs
0,1323,0,7,1,4,3,6,3,71369,17255,...,421,11,4,7,2,13,146312,25898,36,21
1,1317,1,236,14,4,3,5,2,91081,15543,...,15,4,7,0,2,0,83071,9241,10,182
2,932,0,235,4,3,0,4,1,19098,6494,...,82,4,12,1,1,3,67175,6721,12,129
3,2098,0,111,4,14,4,9,11,35821,27058,...,157,4,1,6,7,13,180574,25018,37,217
4,2344,1,145,4,7,7,12,12,269340,33275,...,350,3,7,3,3,34,45127,5891,91,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26843,1468,0,79,4,11,3,6,11,111109,23414,...,2,4,11,6,5,6,151357,28571,35,39
26844,970,0,350,14,3,2,6,17,19435,8042,...,72,4,11,5,0,9,102880,10950,11,39
26845,1480,0,134,7,4,1,3,3,73126,11044,...,141,11,4,2,5,7,129053,21894,31,10
26846,1837,1,59,4,11,3,6,17,122445,31571,...,68,12,4,3,6,7,149570,25492,18,186


In [12]:
match_20_data.to_csv('d:/Final_Project/data/match_20_data.csv',encoding='utf-8-sig')

In [16]:
chall_21_gameId.drop(6847,0,inplace=True)
chall_21_gameId.reset_index(inplace=True)
chall_21_gameId.drop('index',1,inplace=True)
chall_21_gameId

,gameId
0,5353779756
1,5351903351
2,5351960180
3,5351809074
4,5351805859
...,...
13368,5347379013
13369,5347476046
13370,5347434370
13371,5345526209


In [17]:
match_21_data=pd.DataFrame([])
for i in tqdm_notebook(range(len(chall_21_gameId))):
    api_url='https://kr.api.riotgames.com/lol/match/v4/matches/'+str(chall_21_gameId['gameId'][i]) + '?api_key=' + api_key
    r=requests.get(api_url)
    
    while r.status_code!=200: # 요청 제한 또는 오류로 인해 정상적으로 받아오지 않는 상태라면, 5초 간 시간을 지연
        time.sleep(10)
        r = requests.get(api_url)
        
    test=pd.DataFrame(r.json()['participants'])
    test_pre=pd.DataFrame({
    'play_time':r.json()['gameDuration'],
    'result':int(test['stats'][0]['win']),
    'blue_1_champ':test['championId'][0],
    'blue_1_spell1':test['spell1Id'][0],
    'blue_1_spell2':test['spell2Id'][0],
    'blue_1_kill':test['stats'][0]['kills'],
    'blue_1_death':test['stats'][0]['deaths'],
    'blue_1_assist':test['stats'][0]['assists'],
    'blue_1_totaldamage_dealt':test['stats'][0]['totalDamageDealt'],
    'blue_1_totaldamage_taken':test['stats'][0]['totalDamageTaken'],
    'blue_1_visionscore':test['stats'][0]['visionScore'],
    'blue_1_cs':test['stats'][0]['totalMinionsKilled'],   
    'blue_2_champ':test['championId'][1],
    'blue_2_spell1':test['spell1Id'][1],
    'blue_2_spell2':test['spell2Id'][1],
    'blue_2_kill':test['stats'][1]['kills'],
    'blue_2_death':test['stats'][1]['deaths'],
    'blue_2_assist':test['stats'][1]['assists'],
    'blue_2_totaldamage_dealt':test['stats'][1]['totalDamageDealt'],
    'blue_2_totaldamage_taken':test['stats'][1]['totalDamageTaken'],
    'blue_2_visionscore':test['stats'][1]['visionScore'],
    'blue_2_cs':test['stats'][1]['totalMinionsKilled'],    
    'blue_3_champ':test['championId'][2],
    'blue_3_spell1':test['spell1Id'][2],
    'blue_3_spell2':test['spell2Id'][2],
    'blue_3_kill':test['stats'][2]['kills'],
    'blue_3_death':test['stats'][2]['deaths'],
    'blue_3_assist':test['stats'][2]['assists'],
    'blue_3_totaldamage_dealt':test['stats'][2]['totalDamageDealt'],
    'blue_3_totaldamage_taken':test['stats'][2]['totalDamageTaken'],
    'blue_3_visionscore':test['stats'][2]['visionScore'],
    'blue_3_cs':test['stats'][2]['totalMinionsKilled'],
    'blue_4_champ':test['championId'][3],
    'blue_4_spell1':test['spell1Id'][3],
    'blue_4_spell2':test['spell2Id'][3],
    'blue_4_kill':test['stats'][3]['kills'],
    'blue_4_death':test['stats'][3]['deaths'],
    'blue_4_assist':test['stats'][3]['assists'],
    'blue_4_totaldamage_dealt':test['stats'][3]['totalDamageDealt'],
    'blue_4_totaldamage_taken':test['stats'][3]['totalDamageTaken'],
    'blue_4_visionscore':test['stats'][3]['visionScore'],
    'blue_4_cs':test['stats'][3]['totalMinionsKilled'],    
    'blue_5_champ':test['championId'][4],
    'blue_5_spell1':test['spell1Id'][4],
    'blue_5_spell2':test['spell2Id'][4],
    'blue_5_kill':test['stats'][4]['kills'],
    'blue_5_death':test['stats'][4]['deaths'],
    'blue_5_assist':test['stats'][4]['assists'],
    'blue_5_totaldamage_dealt':test['stats'][4]['totalDamageDealt'],
    'blue_5_totaldamage_taken':test['stats'][4]['totalDamageTaken'],
    'blue_5_visionscore':test['stats'][4]['visionScore'],
    'blue_5_cs':test['stats'][4]['totalMinionsKilled'],
    'red_1_champ':test['championId'][5],
    'red_1_spell1':test['spell1Id'][5],
    'red_1_spell2':test['spell2Id'][5],
    'red_1_kill':test['stats'][5]['kills'],
    'red_1_death':test['stats'][5]['deaths'],
    'red_1_assist':test['stats'][5]['assists'],
    'red_1_totaldamage_dealt':test['stats'][5]['totalDamageDealt'],
    'red_1_totaldamage_taken':test['stats'][5]['totalDamageTaken'],
    'red_1_visionscore':test['stats'][5]['visionScore'],
    'red_1_cs':test['stats'][5]['totalMinionsKilled'],
    'red_2_champ':test['championId'][6],
    'red_2_spell1':test['spell1Id'][6],
    'red_2_spell2':test['spell2Id'][6],
    'red_2_kill':test['stats'][6]['kills'],
    'red_2_death':test['stats'][6]['deaths'],
    'red_2_assist':test['stats'][6]['assists'],
    'red_2_totaldamage_dealt':test['stats'][6]['totalDamageDealt'],
    'red_2_totaldamage_taken':test['stats'][6]['totalDamageTaken'],
    'red_2_visionscore':test['stats'][6]['visionScore'],
    'red_2_cs':test['stats'][6]['totalMinionsKilled'],
    'red_3_champ':test['championId'][7],
    'red_3_spell1':test['spell1Id'][7],
    'red_3_spell2':test['spell2Id'][7],
    'red_3_kill':test['stats'][7]['kills'],
    'red_3_death':test['stats'][7]['deaths'],
    'red_3_assist':test['stats'][7]['assists'],
    'red_3_totaldamage_dealt':test['stats'][7]['totalDamageDealt'],
    'red_3_totaldamage_taken':test['stats'][7]['totalDamageTaken'],
    'red_3_visionscore':test['stats'][7]['visionScore'],
    'red_3_cs':test['stats'][7]['totalMinionsKilled'],
    'red_4_champ':test['championId'][8],
    'red_4_spell1':test['spell1Id'][8],
    'red_4_spell2':test['spell2Id'][8],
    'red_4_kill':test['stats'][8]['kills'],
    'red_4_death':test['stats'][8]['deaths'],
    'red_4_assist':test['stats'][8]['assists'],
    'red_4_totaldamage_dealt':test['stats'][8]['totalDamageDealt'],
    'red_4_totaldamage_taken':test['stats'][8]['totalDamageTaken'],
    'red_4_visionscore':test['stats'][8]['visionScore'],
    'red_4_cs':test['stats'][8]['totalMinionsKilled'],
    'red_5_champ':test['championId'][9],
    'red_5_spell1':test['spell1Id'][9],
    'red_5_spell2':test['spell2Id'][9],
    'red_5_kill':test['stats'][9]['kills'],
    'red_5_death':test['stats'][9]['deaths'],
    'red_5_assist':test['stats'][9]['assists'],
    'red_5_totaldamage_dealt':test['stats'][9]['totalDamageDealt'],
    'red_5_totaldamage_taken':test['stats'][9]['totalDamageTaken'],
    'red_5_visionscore':test['stats'][9]['visionScore'],
    'red_5_cs':test['stats'][9]['totalMinionsKilled']
    },index=[i])
    match_21_data=pd.concat([match_21_data,test_pre])
match_21_data.to_csv('D:/Final_Project/data/match_21_data.csv',encoding='utf-8-sig')
match_21_data

<ipython-input-17-5e208a30990c>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(chall_21_gameId))):


,play_time,result,blue_1_champ,blue_1_spell1,blue_1_spell2,blue_1_kill,blue_1_death,blue_1_assist,blue_1_totaldamage_dealt,blue_1_totaldamage_taken,...,red_5_champ,red_5_spell1,red_5_spell2,red_5_kill,red_5_death,red_5_assist,red_5_totaldamage_dealt,red_5_totaldamage_taken,red_5_visionscore,red_5_cs
0,2241,0,7,12,4,4,5,18,156358,37416,...,64,11,4,9,8,11,143137,51158,47,31
1,1022,0,13,4,12,0,1,0,73834,6213,...,111,14,4,2,0,9,20522,4193,32,35
2,2056,1,36,12,4,1,5,11,148044,32263,...,7,12,4,10,7,9,139167,31081,28,164
3,1378,1,777,12,4,11,4,4,152755,17974,...,48,11,4,2,8,4,80229,30363,31,16
4,1265,1,523,1,4,5,1,15,108625,12931,...,142,14,4,2,3,4,73738,8879,21,135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13368,1363,1,412,14,4,2,4,12,16639,8032,...,134,4,12,2,2,5,85185,11636,8,166
13369,1553,1,39,4,12,10,4,2,213285,23667,...,58,4,12,2,3,2,108021,29559,24,193
13370,1143,1,115,4,12,5,2,8,102764,6862,...,240,14,4,0,7,2,22963,20706,11,50
13371,1261,0,360,1,4,4,6,3,80037,12581,...,115,12,4,17,2,7,154117,8985,27,168
